In [1]:
from collections import deque

## Part One

In [2]:
def parse(s):
    files = []
    free_blocks = deque()
    
    for id, file_size in enumerate(s[::2]):
        files.append((id, int(file_size)))
        
    for free_size in s[1::2]:
        free_blocks.append(int(free_size))
        
    return files, free_blocks

def compact(files, free_blocks):
    free_blocks = deque(free_blocks)
    files = deque(files)
    compact_id, compact_count = files.pop()
    pos = 0
    checksum = 0
    while files:
        try:
            file_id, size = files.popleft()
            for n in range(size):
                checksum += file_id * pos
                pos += 1
        except IndexError:
            break

        free = free_blocks.popleft()
        for n in range(free):
            checksum += compact_id * pos
            pos += 1
            compact_count -= 1
            if compact_count < 1:
                try:
                    compact_id, compact_count = files.pop()
                except IndexError:
                    break
    # straglers:
    for i in range(compact_count):
        checksum += pos * compact_id
        pos += 1        
    
    return checksum

In [3]:
s="2333133121414131402"
     
files, free_blocks = parse(s)
compact(files, free_blocks)

1928

In [4]:
with open('input_files/09.txt') as f:
    s = f.read()

files, free_blocks = parse(s)
print("part one: ", compact(files, free_blocks))

part one:  6323641412437


## Part Two

In [7]:
def compact_whole_files(files, free_blocks):
    # move blocks in next slot with enough space
    # Keep track of remaining space and moved ids
    disk_blocks = [(space, []) for space in free_blocks]
    
    for back_index, (file_id, file_size) in enumerate(reversed(files)):
        # get the next block with available space
        # careful not to find space after this block!
        next_free = next(( (i, block) 
                          for i, block in enumerate(disk_blocks[:len(disk_blocks) - back_index]) 
                          if block[0] >= file_size), None)
        if next_free is not None:
            i, (block_size, members) = next_free
            disk_blocks[i] = (block_size - file_size, members + [file_id] * file_size)
            files[-(back_index+1)] = (0, file_size)

    # calculate checksum
    pos = 0
    checksum = 0

    for original, [remaining, moved] in zip(files, disk_blocks):
        id, size = original
        for _ in range(size):
            checksum += pos * id
            pos += 1
        for id in moved:
            checksum += pos * id
            pos += 1
        pos += remaining
    
    return checksum

In [8]:
s="2333133121414131402"
  
files, free_blocks = parse(s)
compact_whole_files(files, free_blocks)

2858

In [9]:
with open('input_files/09.txt') as f:
    s = f.read()
files, free_blocks = parse(s)

print("part two:", compact_whole_files(files, free_blocks))

part two: 6351801932670
